### Giveme5W1H 코드 해체
출처 : https://github.com/fhamborg/Giveme5W1H
- 목적 :
    * PROMED 기사에서 육하원칙을 룰 기반으로 추출해내는 것
    *  Giveme5W1H의 원리를 사용(후보군 추출 후 점수)
- 그 중에서도 cause extractor를 사용하여서 질병의 원인, 감염원, 증상 등을 추출하는 것을 목적으로 한다
- 나머지 육하원칙은 그리 중요하지는 않음

In [2]:
#예문
text = ' A vegetable farm in Chongpyong County [South Hamgyong province] suffered from poor crop yields due to pests and disease. The farm produces cabbage and cucumber, but the cabbage began rotting from the roots and falling apart due to damage from an unidentified disease, along with pests. The farm was only able to meet 40% of its production target.The farmers attempted to salvage the situation on their own, but were ultimately unable to do so. [They] requested that authorities find ways to resolve the problem, noting that even if the rotting cabbage was quickly harvested and delivered to Pyongyang, it would likely be completely rotten by the time it arrived.The authorities quickly formed a team of "plant disease control experts" [which] discovered that instead of purple cabbage seeds, the farm had planted regular large cabbage seeds. This led to disease and pests appearing on the roots. The farm already knew that only the purple seeds were suitable for the soil, [but] were unable to acquire any of the right seeds last year [2019].Farmers are reportedly in the process of fumigating their facilities and turning the rotten cabbage into compost.'

#### 기존 Giveme5W1H의 사용법

In [3]:
#1. Standford-NLP Server를 구동해야 함
# stanford-corenlp-full-2017-06-09.jar 파일 다운로드 후, runtime-resources 폴더에 압축 풀기
# cd C:\Users\ms.kim1\AppData\Local\Programs\Python\Python36\Lib\site-packages\Giveme5W1H\runtime-resources\stanford-corenlp-full-2017-06-09
# java -mx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 15000

In [4]:
from Giveme5W1H.extractor.document import Document
from Giveme5W1H.extractor.extractor import MasterExtractor

extractor = MasterExtractor() #extractor object 만들기
doc = Document.from_text(text) #document object 만들기
# or: doc = Document(title, lead, text, date_publish) 
doc = extractor.parse(doc) #parse의 기능 : 전처리 -> 육하원칙 후보 구문 추출 -> 점수 매기기 -> 각 분야에서 가장 높은 점수를 얻은 구문을 답으로 반환

top_who_answer = doc.get_top_answer('why').get_parts_as_text() #why : the farm
top_who_answer = doc.get_top_answer('how').get_parts_as_text() #how : from poor crop yields due to pests and disease .
print('how : '+top_who_answer)

how : from poor crop yields due to pests and disease .


그러나, 그리 만족스러운 답을 얻지는 못함. 코드를 해체해서 커스터마이징을 하기로 함

#### PreProcess 과정 이용하기
- 먼저, how, why 등과 같은 요소를 얻고 싶기 때문에 cause_extractor를 커스터마이징하기로 결정

##### 만났던 문제점
- cause_extractor 상에서 document.get_trees()란 메소드를 발견, document.py 상에서는 그냥 하나의 오브젝트를 초기화하거나 call하는 정도의 기능만 있음.
- 알고보니, tree란 구조는 stardford corenlp server를 통해 나온 결과의 포맷이었음.
- 즉, cause_extractor object를 사용하려면 tree구조의 document를 얻어야 했음

In [5]:
#필요 패키지 선언
from Giveme5W1H.extractor.document import Document
from Giveme5W1H.extractor.preprocessors.preprocessor_core_nlp import Preprocessor

In [6]:
doc = Document.from_text(text) #document 객체 생성
pre_text=Preprocessor() #Preprocessor 객체 만들기
doc2=pre_text.preprocess(doc) #전처리 객체에서 document객체를 이용해서 전처리 method 사용

In [7]:
type(doc2)

NoneType

왜인지, 제대로 적용이 되지 않음. 이는 아직 코드의 구조에 대해 제대로 이해하지 못해서 method가 정확하게 call이 되지 않은 것으로 보임

#### 직접 standford server nlp를 사용하여 전처리를 수행하여 tree구조를 구하기로 결정
- 뒷 과정은 함수를 그냥 하나하나 다 뜯어서 활용

In [8]:
#1. Standford-NLP Server를 구동해야 함
# stanford-corenlp-full-2017-06-09.jar 파일 다운로드 후, runtime-resources 폴더에 압축 풀기
# cd C:\Users\ms.kim1\AppData\Local\Programs\Python\Python36\Lib\site-packages\Giveme5W1H\runtime-resources\stanford-corenlp-full-2017-06-09
# java -mx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 15000

In [9]:
#2. 필요한 패키지 선언
import nltk
from pycorenlp import StanfordCoreNLP

In [10]:
#3. annotation획득
nlp=StanfordCoreNLP('http://localhost:9000')
actual_config=base_config = {
            'timeout': 500000,
            'annotators': 'tokenize,ssplit,pos,lemma,parse,ner,depparse,mention,coref',
            'tokenize.language': 'English',
            'outputFormat': 'json'
        }
res=nlp.annotate(text,actual_config)
#Giveme5W1H에서 선언한 configuration요소들

In [11]:
# res['sentences'][0] 
# 각 문장별로, index, parse, position, coref 등의 요소 반환해줌(json 형식으로, dictionary 타입임)

In [30]:
#4. tree구조를 만들기로 함
annotation = res
tree=[] 
for sentence in annotation['sentences']:
    _token_index = 0
    _tokens = sentence['tokens']
    sentence_tree=nltk.Tree.fromstring(sentence['parse'])
    # sentence_tree = nltk.ParentedTree.fromstring(sentence['parse']) #read_leaf 함수가 없어서 삭제한 버전으로, 나중에 evaluate tree함수 부분에서 pos가 반환이 안됨
    # sentence_tree = nltk.ParentedTree.fromstring(sentence['parse'],read_leaf=_link_leaf_to_core_nlp()) #pos가 나오게 하는 read_leaf 사용하기
    # add a reference to the original data from parsing for this sentence
    sentence_tree.stanfordCoreNLPResult = sentence

    tree.append(sentence_tree)

이 때, token요소만 따로 추출해서, 따로 선언했던 _link_leaf_to_core_nlp에 인자로 던져주는 로직이 있었고, 그것의 결과를 tree 구조를 구할 때 read_leaf의 인자로 반환하는 코드가 있었다. 확인해본 결과, read_leaf는 반환되는 tree구조의 쓰이는 양식으로 보임. 또, _link_leaf_to_core_nlp는 문장의 token관련 요소의 값에 따라 반환되는 그 형식과 결과가 달라졌다. token의 길이가 1보다 작은 경우와 큰 경우로 나누어져 있었다. read_leaf에 result를 따로 저장하여 입력했지만, 계속 error가 나오는 관계로 일단 제쳐두고 tree구조의 결과값을 받았다. 
(Giveme5W1H > extractor > preprocessors > preprocessor_core_nlp.py > _link_leaf_to_core_nlp함수)

In [31]:
coerfs = annotation['corefs']
#문장 속 단어간의 상관계수를 저장하는 변수로 보임

In [32]:
tokens = [] #단어의 토큰화 : 문장 속 단어를 쪼개는 작업
pos = []  # 문장 속 단어의 위치
ner = [] # 개체명인식 : 문맥상 의미를 파악하여 entity를 추출하여 미리 정의된 분류체계로 정보를 분류하는 것

for sentence in annotation['sentences']:
    s_tokens = []
    s_pos = []
    s_ner = []
    for token in sentence['tokens']:
        s_tokens.append(token)
        s_pos.append((token['originalText'], token['pos'])) #원문 + 위치
        s_ner.append((token['originalText'], token['ner'])) #원문 + ner

    tokens.append(s_tokens)
    pos.append(s_pos)
    ner.append(s_ner)

tree구조 이외로도 필요한 요소들을 따로 저장해두는 작업을 진행.(coerfs, tokens, position, NER)

#### PreProcessing 과정 후 cause_extractor 부분을 실행하면서 그 결과가 어떻게 나온 것인지 확인하기로 함
- 본격적으로 tree구조를 사용하여 바로 cause를 뽑아내기 전, 개발자가 원인을 뽑아내기 위해 구축한 간단한 사전들을 온톨로지를 활용하여 쪼개놓는 작업을 먼저 해야 함

In [33]:
#1. Frinston에서 제작한 영어 단어 온톨로지 wordnet을 활용/필요 패키지 다운 및 선언
# import nltk
# nltk.download('wordnet') 

from nltk.corpus import wordnet

In [34]:
#2. 개발자가 선언한 casual_verbs : 원인과 보통 함께 쓰이는 동사를 모음
causal_verbs = ['activate', 'actuate', 'arouse', 'associate', 'begin', 'bring', 'call', 'cause', 'commence',
                    'conduce', 'contribute', 'create', 'derive', 'develop', 'educe', 'effect', 'effectuate', 'elicit',
                    'entail', 'evoke', 'fire', 'generate', 'give', 'implicate', 'induce', 'kick', 'kindle', 'launch',
                    'lead', 'link', 'make', 'originate', 'produce', 'provoke', 'put', 'relate', 'result', 'rise', 'set',
                    'spark', 'start', 'stem', 'stimulate', 'stir', 'trigger', 'unleash']

# translate causal verbs into synsets(유의어 추출)
synsets = []
for verb in causal_verbs:
    synsets += wordnet.synsets(verb, 'v')
    causal_verbs = set(synsets)

In [35]:
# verbs involved in NP-VP-NP constraints(문법 형식에 따라 다르게 나타나는 동사군)
constraints_verbs = {'cause': None, 'associate': None, 'relate': None, 'lead': None, 'induce': None}

for verb in constraints_verbs:
    constraints_verbs[verb] = set(wordnet.synsets(verb, 'v'))

In [36]:
#hyponym:하위어/
def get_hyponyms(synsets):
    """
    Fetches all hyponyms in a recursive manner creating a word class.

    :param synsets: The list of synsets to process
    :type synsets: [synset]

    :return: A set of synsets
    """

    result = set()
    for hyponym in synsets.hyponyms():
        result |= get_hyponyms(hyponym) #|= : 비트or 연산 후 할당/0 0 -> 0, 0 1 -> 1, 1 1 -> 1
    return result | set(synsets.hyponyms())


In [37]:
# hyponym classes involved in NP-VP-NP constraints
# hyponym : ~~에 속하는, 목, 속 
constraints_hyponyms = {'entity': None, 'phenomenon': None, 'abstraction': None, 'group': None, 'possession': None,
                        'event': None, 'act': None, 'state': None}
# initialize synsets that are used as constraints in NP-VP-VP patterns
for noun in constraints_hyponyms:
    hyponyms = set()
    for synset in wordnet.synsets(noun, 'n'):
        hyponyms |= get_hyponyms(synset)
    constraints_hyponyms[noun] = hyponyms

In [38]:
from nltk.stem.wordnet import WordNetLemmatizer
#형태소분석기 : 공통기저형인 어휘소/어간
lemmatizer = WordNetLemmatizer()

In [39]:
weights = (.56,.44,.27,.026) #evaluate하기 위한 가중치 set

원인에 해당할만한 구문을 추출하기 위한 코드. 이 때 tree 구조를 평가한 결과를 기반으로 구문 후보를 뽑아내기 때문에, tree 평가 함수를 사용하도록 한다.

In [53]:
# Extract Candidate
"""
Extracts possible agents/actions pairs from a given document.
Candidates are chosen if they belong to an coref-chain and is part of a NP-VP-NP pattern

:param document: The Document to be analyzed.
:type document: Document

:return: A List of Tuples containing all agents, actions and their position in the document.
"""
candidates = []
postrees = tree

from Giveme5W1H.extractor.candidate import Candidate

for i, tree in enumerate(postrees):
    for candidate in _evaluate_tree(tree):
        candidateObject = Candidate()
        # used by the extractor
        candidateObject.set_raw(candidate[0])  # candidate[0] contains the cause, candidate[1] the effect
        candidateObject.set_type(candidate[2])
        candidateObject.set_sentence_index(i)

        candidates.append(candidateObject)

NameError: name 'self' is not defined

In [ ]:
def _evaluate_tree(tree): #추출한 tree를 평가하는 부분!!!
        """
        Determines if the given sub tree contains a cause/effect relation.

        The indicators used in this function are inspired by:
        "Automatic Extraction of Cause-Effect Information from Newspaper Text Without Knowledge-based Inferencing"
        by Khoo et. al. (adverbs + conjunctions)
        "Automatic Detection of Causal Relations for Question Answering" by Roxana Girj (verbs)

        :param tree: A tree to analyze
        :type tree: ParentedTree

        :return: A Tuple containing the cause/effect phrases and the pattern used to find it.
        """
        _candidatesObjects = []
        candidates = []
        pos = tree.pos() #tree 구조 안에 있는 요소로, 단어의 문장 내 위치를 의미 / read_leaf를 지정해주지 않으면, list형식이어서 pos가 먹히지 않음
        tokens = [t[0] for t in pos]

        # Searching for cause-effect relations that involve a verb/action we look for NP-VP-NP
        for subtree in tree.subtrees(filter=lambda t: t.label() == 'NP' and t.right_sibling() is not None):
            sibling = subtree.right_sibling()

            # skip to the first verb
            while sibling.label() == 'ADVP' and sibling.right_sibling() is not None:
                sibling = sibling.right_sibling()

            # NP-VP-NP pattern found .__repr__()
            if sibling.label() == 'VP' and "('NP'" in sibling.__repr__():
                verbs = [t[0] for t in sibling.pos() if t[1][0] == 'V'][:3]
                verb_synset = set()

                # depending on the used tense, we may have to look at the second/third verb e.g. 'have been ...'
                for verb in verbs:
                    normalized = verb['nlpToken']['originalText'].lower()

                    # check if word meaning is relevant
                    verb_synset = set(wordnet.synsets(normalized, 'v'))
                    if verb_synset.isdisjoint(self.causal_verbs):
                        continue

                    # if necessary look at the  following phrase
                    lemma = self.lemmatizer.lemmatize(normalized)
                    if lemma in self.causal_verb_phrases:
                        # fetch following two tokens
                        rest = ''
                        for i, token in enumerate(verbs):
                            if verb['nlpToken']['word'] == token['nlpToken']['word']:
                                rest = ' '.join([t['nlpToken']['word'] for t in verbs[i + 1:i + 3]]).lower()
                                break
                        if rest != self.causal_verb_phrases[lemma]:
                            continue

                # According to Girju, if the found verb is 'cause' or a synonym, the following NP is 100% the cause
                # so we can directly put it in the list of candidates
                if not verb_synset.isdisjoint(self.constraints_verbs['cause']):
                    candidates.append(deepcopy([subtree.pos(), sibling.pos(), 'NP-VP-NP']))
                else:
                    # pattern contains a valid verb (that is not 'cause'), so check the 7 subpatterns
                    pre = [t[0]['nlpToken']['originalText'].lower() for t in subtree.pos() if
                           t[1][0] == 'N' and t[0]['nlpToken']['originalText'].isalpha()]
                    post = [t[0]['nlpToken']['originalText'].lower() for t in sibling.pos() if
                            t[1][0] == 'N' and t[0]['nlpToken']['originalText'].isalpha()]
                    pre_con = {'entity': False, 'abstraction': False}
                    post_con = {'entity': False, 'phenomenon': False, 'abstraction': False, 'group': False,
                                'possession': False, 'event': False, 'act': False, 'state': False}
                    verb_con = {'associate': False, 'relate': False, 'lead': False, 'induce': False}

                    # check nouns in after verb
                    for noun in post:
                        noun_synset = set(wordnet.synsets(noun, 'n'))
                        for con in post_con:
                            post_con[con] = post_con[con] or not noun_synset.isdisjoint(self.constraints_hyponyms[con])
                            if post_con['phenomenon']:
                                break

                        if post_con['phenomenon']:
                            break

                    # check nouns in before verb
                    for noun in pre:
                        noun_synset = set(wordnet.synsets(noun, 'n'))
                        for con in pre_con:
                            pre_con[con] = pre_con[con] or not noun_synset.isdisjoint(self.constraints_hyponyms[con])

                    # check if verb is relevant for a subpattern
                    for con in verb_con:
                        verb_con[con] = not verb_synset.isdisjoint(self.constraints_verbs[con])
                        if verb_con[con]:
                            break

                    # apply subpatterns
                    if (
                            post_con['phenomenon']
                    ) or (
                            not pre_con['entity'] and (verb_con['associate'] or verb_con['relate']) and (
                            post_con['abstraction'] and post_con['group'] and post_con['possession'])
                    ) or (
                            not pre_con['entity'] and post_con['event']
                    ) or (
                            not pre_con['abstraction'] and (post_con['event'] or post_con['act'])
                    ) or (
                            verb_con['lead'] and (not post_con['entity'] and not post_con['group'])
                    ):
                        candidates.append(deepcopy([subtree.pos(), sibling.pos(), 'NP-VP-NP']))

        # search for adverbs or clausal conjunctions
        for i in range(len(tokens)):
            token = tokens[i]['nlpToken']['originalText'].lower()

            if pos[i][1] == 'RB' and token in self.adverbial_indicators:
                # If we come along an adverb (RB) check the adverbials that indicate causation
                candidates.append(deepcopy([pos[:i], pos[i - 1:], 'RB']))

            elif token in self.causal_conjunctions and ' '.join(
                    [x['nlpToken']['originalText'] for x in tokens[i:]]).lower().startswith(
                    self.causal_conjunctions[token]):
                # Check if token is a clausal conjunction indicating causation
                start = i
                if token not in self.causal_conjunctions_inclusive:
                    # exclude clausal conjunction besides special cases
                    start += 1
                candidates.append(deepcopy([pos[start:], pos[:i], 'biclausal']))

        # drop candidates containing other candidates
        unique_candidates = []
        candidate_strings = []
        for candidate in candidates:
            # Bugfix, at some very rare occasions, the candidate holds an empty list
            if len(candidate[0]) > 0:
                another_string = [x[0]['nlpToken']['originalText'] for x in candidate[1]]
                a_string = candidate[0][0][0]['nlpToken']['originalText'] + ' ' + ' '.join(another_string)
                candidate_strings.append(a_string)

        for i, candidate in enumerate(candidates):
            unique = True
            for j, substring in enumerate(candidate_strings):
                if i != j and candidate[2] == candidates[j][2] and substring in candidate_strings[i]:
                    unique = False
                    break
            if unique:
                unique_candidates.append(candidate)

        return unique_candidates


In [ ]:
tree.subtrees(filter=lambda t: t.label() == 'NP' and t.right_sibling() is not None)